In [1]:
import sys

sys.path.insert(0, '../')
import utils.common as cm
import utils.bootcampviztools as btz

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv(cm.DATA_PATH)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   vehicle_type            100000 non-null  object 
 1   make                    100000 non-null  object 
 2   model                   100000 non-null  object 
 3   version                 99842 non-null   object 
 4   fuel                    96810 non-null   object 
 5   year                    100000 non-null  int64  
 6   kms                     99189 non-null   float64
 7   power                   99709 non-null   float64
 8   doors                   0 non-null       float64
 9   shift                   98982 non-null   object 
 10  color                   0 non-null       float64
 11  photos                  100000 non-null  int64  
 12  description             55810 non-null   object 
 13  price                   100000 non-null  int64  
 14  currency             

# Descripción de las variables
<table>
<tr>
    <th>Nombre de columna</th>
    <th>Tipo de variable</th>
    <th>Descripción</th>
    <th>Comentarios</th>
</tr>
<tr>
    <td>vehicle_type</td>
    <td>Categórica</td>
    <td>Tipo de vehículo</td>
    <td>Todos los vehículos son coches (columna para eliminar)</td>
</tr>
<tr>
    <td>make</td>
    <td>Categórica</td>
    <td>Marca del vehículo</td>
    <td></td>
</tr>
<tr>
    <td>model</td>
    <td>Categórica</td>
    <td>Modelo del vehículo</td>
    <td>Habrá que ver si los nombres están bien escritos o hay valores repetidos</td>
</tr>
<tr>
    <td>version</td>
    <td>Texto/categórica</td>
    <td>Versión del modelo concreto de coche</td>
    <td>Habrá que valorar si se puede sacar información interesante de esa columna</td>
</tr>
<tr>
    <td>fuel</td>
    <td>Categórica</td>
    <td>Tipo de combustible que usa el vehículo</td>
    <td></td>
</tr>
<tr>
    <td>year</td>
    <td>Numérica</td>
    <td>Año al que pertenece el vehículo (fabricación)</td>
    <td></td>
</tr>
<tr>
    <td>kms</td>
    <td>Numérica</td>
    <td>Kilometraje realizado por el vehículo</td>
    <td></td>
</tr>
<tr>
    <td>power</td>
    <td>Numérica</td>
    <td>Potencia en caballos que tiene el coche</td>
    <td></td>
</tr>
<tr>
    <td>doors</td>
    <td>Numérica</td>
    <td>Número de puertas</td>
    <td>Tiene un 100% de missings, en principio se eliminará (a no ser que en otra columna encontremos el número de puertas)</td>
</tr>
<tr>
    <td>shift</td>
    <td>Binaria</td>
    <td>Tipo de cambio</td>
    <td>Manual; Automatic</td>
</tr>
<tr>
    <td>color</td>
    <td>Numérica</td>
    <td>Color del coche</td>
    <td>Tiene un 100% de missings, en principio se eliminará esta columna</td>
</tr>
<tr>
    <td>photos</td>
    <td>Numérica</td>
    <td>Número de fotos subidas al anuncio del coche</td>
    <td></td>
</tr>
<tr>
    <td>description</td>
    <td>Texto</td>
    <td>Anuncio del coche</td>
    <td>Probablemente eliminemos esta columna (44% de missings)</td>
</tr>
<tr>
    <td>price</td>
    <td>Numérica</td>
    <td>Precio del coche</td>
    <td>Esta columna será nuestro TARGET</td>
</tr>
<tr>
    <td>currency</td>
    <td>Categórica</td>
    <td>Tipo de moneda en la que se encuentra el precio</td>
    <td>Columna a eliminar, ya que tiene un único valor: €</td>
</tr>
<tr>
    <td>location</td>
    <td>Categórica</td>
    <td>Población en la que se vende el vehículo</td>
    <td>Valoraremos si utilizarla o no (tenemos código postal)</td>
</tr>
<tr>
    <td>publish_date</td>
    <td>Datetime</td>
    <td>Fecha de publicación del anuncio</td>
    <td></td>
</tr>
<tr>
    <td>update_date</td>
    <td>Datetime</td>
    <td>Fecha de actualización del anuncio</td>
    <td></td>
</tr>
<tr>
    <td>dealer_name</td>
    <td>Texto</td>
    <td>Nombre del vendedor particular o empresa</td>
    <td></td>
</tr>
<tr>
    <td>dealer_description</td>
    <td>Texto</td>
    <td>Descripción de la empresa de venta</td>
    <td>Probablemente eliminemos esta columna</td>
</tr>
<tr>
    <td>dealer_address</td>
    <td>Texto</td>
    <td>Dirección donde se encuentra la empresa que vende el vehículo</td>
    <td>Probablemente eliminemos esta columna</td>
</tr>
<tr>
    <td>dealer_zip_code</td>
    <td>Numérica</td>
    <td>Código postal del vendedor</td>
    <td></td>
</tr>
<tr>
    <td>dealer_city</td>
    <td>Texto</td>
    <td>Ciudad en la que se encuentra el vendedor</td>
    <td></td>
</tr>
<tr>
    <td>dealer_country_code</td>
    <td>Categórica</td>
    <td>Código escrito del nombre del país</td>
    <td>Esta columna tiene un único valor, la eliminaremos</td>
</tr>
<tr>
    <td>dealer_is_professional</td>
    <td>Binaria</td>
    <td>Indica si el vendedor es una empresa profesional que se dedica a ello o un particular</td>
    <td>True: profesional; False: particular</td>
</tr>
<tr>
    <td>dealer_website</td>
    <td>Texto</td>
    <td>Link a la página web de la empresa que vende el vehículo</td>
    <td>Esta columna la elimnaremos</td>
</tr>
<tr>
    <td>dealer_registered_at</td>
    <td>Datetime</td>
    <td>Fecha en la que el vendedor se registró en la plataforma</td>
    <td>Probablemente eliminaremos esta columna</td>
</tr>
<tr>
    <td>date</td>
    <td>Datetime</td>
    <td>Fecha en la que se extrajo la información</td>
    <td>Valoraremos si eliminar esta columna o solo utilizar una fecha concreta</td>
</tr>
</table>

In [4]:
total_cols = list(df.columns)

In [5]:
cm.data_report(df)

,DATA_TYPE,MISSINGS (%),UNIQUE_VALUES,CARDIN (%)
COL_N,,,,
vehicle_type,object,0.00,1,0.00
make,object,0.00,66,0.07
model,object,0.00,748,0.75
version,object,0.16,8658,8.66
fuel,object,3.19,4,0.00
year,int64,0.00,43,0.04
kms,float64,0.81,6643,6.64
power,float64,0.29,349,0.35
doors,float64,100.00,0,0.00


In [8]:
df.sample(10)

,vehicle_type,make,model,version,fuel,year,kms,power,doors,shift,...,dealer_name,dealer_description,dealer_address,dealer_zip_code,dealer_city,dealer_country_code,dealer_is_professional,dealer_website,dealer_registered_at,date
18636,car,Citroen,C4 Cactus,PureTech 82cv S&S ETG Feel - 5 P (2015),Gasolina,2015,109000.0,82.0,NaN,automatic,...,FLEXICAR A.,"Para Comprar o Vender tu coche, Primero en FLE...","Colegio Privado Don Bosco Salesianos, avenida ...",3010.0,Alicante/Alacant,ES,True,www.flexicar.es,2021-05-11,2023-02-06
41682,car,Alfa Romeo,Stelvio,2.2 JTDM 210 Veloce Q4 5p S/S Aut.,Diésel,2020,40000.0,210.0,NaN,automatic,...,Ignacio E.,Concesionario multimarca en Las Rozas\nC/ ROTT...,"Calle Rotterdam, Parque Industrial Tecnológico...",28230.0,Las Rozas de Madrid,ES,True,www.exclusivemotorgallery.com,2018-10-24,2023-02-06
79733,car,Nissan,Micra,1.5 dCi SANDS N-Connecta 66 kW (90 CV),Diésel,2017,62183.0,90.0,NaN,manual,...,OCASIONPLUS T.,"OcasionPlus, el líder nacional en la compraven...","Peugeot Auto Rallye Torrejón, Avenida de la Co...",28851.0,Mercado Parque Corredor del Henares,ES,True,www.ocasionplus.com,2022-10-03,2023-01-26
54674,car,Opel,Grandland X,1.2 TURBO ULTIMATE AUTO,Gasolina,2020,55000.0,130.0,NaN,NaN,...,Gedescoche B.,¿Necesitas un coche y no obtienes financiación...,"299, Avinguda Diagonal, el Fort Pienc, Eixampl...",8009.0,Barcelona,ES,True,https://www.financiaciondecoches.es/,2021-10-11,2023-01-30
1516,car,Volvo,XC70,2.0 D4 181 Momentum 5p Aut.,Diésel,2016,250000.0,181.0,NaN,automatic,...,ALSAN AUTOMOVILES ..,NaN,"Polígono Industrial Torrehierro, Talavera de l...",45613.0,Gamonal,ES,True,NaN,2016-03-22,2023-01-30
80575,car,Renault,Kadjar,Zen Energy TCe 97 kW (130 CV),Gasolina,2018,80945.0,130.0,NaN,manual,...,OcasionPlus M.,NaN,"BV-5031, Avinguda de Nostra Senyora de Montser...",8394.0,Sant Vicenç de Montalt,ES,True,NaN,2022-12-19,2023-01-26
81029,car,Land Rover,Range Rover Sport,3.0 TDV6 HSE 190 kW (258 CV),Diésel,2017,111286.0,258.0,NaN,automatic,...,OCASIONPLUS T.,"OcasionPlus, el líder nacional en la compraven...","Peugeot Auto Rallye Torrejón, Avenida de la Co...",28851.0,Mercado Parque Corredor del Henares,ES,True,www.ocasionplus.com,2022-10-03,2023-02-02
21470,car,Ford,Puma,1.0 EcoBoost MHEV Titanium 92 kW (125 CV),Gasolina,2020,39150.0,125.0,NaN,manual,...,OCASION P.,El primer gran centro de OcasionPlus en Barcel...,"Passeig de Gràcia, la Dreta de l'Eixample, Eix...",8007.0,Barcelona,ES,True,www.ocasionplus.com,2019-10-31,2023-01-26
23458,car,Citroen,DS3,1.2 VTi 82 Design 3p,Gasolina,2014,107540.0,82.0,NaN,manual,...,BAHIAMOCION ..,Bahiamoción es Servicio Oficial Seat en Rota (...,"Puntal de Levante, Rota, Costa Noroeste, Cádiz...",11520.0,Rota,ES,True,https://www.bahiamocion.es/,2018-10-18,2023-02-06
8431,car,Mitsubishi,3000 GT,3.0 286 3p,Gasolina,1993,182300.0,150.0,NaN,manual,...,Optimecar V.,"Somos una empresa de capital nacional, que des...","1, Calle Pascal, Churriana, Málaga, Málaga-Cos...",29004.0,Málaga,ES,True,www.optimecar.com,2017-10-04,2023-01-27
